In [1]:
import pandas as pd
import numpy as np
from math import sqrt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from scipy.spatial.distance import pdist, squareform, hamming

def import_data_encoded():
    db = pd.read_csv('dataset/db_modelli_smiles_pubchem.csv').drop(columns = 'Unnamed: 0')
    # codifica
    db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

    db = db.drop(columns = 'test_cas')
    # print(db.info())

    # Ordinal Encoding
    encoder = OrdinalEncoder(dtype = int)

    encoder.fit(db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])

    db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']] = encoder.transform(
        db[['species', 'conc1_type', 'exposure_type', 'class', 'tax_order', 'family', 'genus']])+1
    
    # Divido il dataset completo tra dati e target
    X = db.drop(columns = 'conc1_mean')
    y = db['conc1_mean'].values

    # splitting
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # ricongiungo train con test
    X_try = X_train.append(X_test)

    # tengo traccia della lunghezza del train set
    len_X_train = len(X_train) 

    return X_try, X_train, X_test, y_train, y_test, len_X_train

X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded()

In [2]:
from mem_distance_matrix import *

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']
non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

X_train_new, X_test_new = fast_dist_mat(X_try, len_X_train, categorical, non_categorical,
                                               alphas = [0.0016102620275609393, 0.5, 0.0016102620275609393, 0],
                                               choice = [1,0])

START fast...
You choose Hamming 1, Euclidean 2 and Hamming on pubchem2d 3...
Start Hamming su categorical...
Start Euclidean...
Start Hamming su Pubchem2d...
...FINISH


In [3]:
neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors = 1, leaf_size = 70)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)
print('Model: {}, \n Accuracy: {},\n RMSE: {}'.format(neigh,
                                                      accuracy_score(y_test, y_pred),
                                                      sqrt(mean_squared_error(y_test, y_pred))))

Model: KNeighborsClassifier(algorithm='auto', leaf_size=70, metric='precomputed',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform'), 
 Accuracy: 0.9008704690139016,
 RMSE: 0.31484842541467234


In [2]:
from mem_distance_matrix import *

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']
non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

X_train_new, X_test_new = fast_dist_mat(X_try, len_X_train, categorical, non_categorical,
                                               alphas = [0.0016102620275609393, 0.2, 0.0016102620275609393, 0],
                                               choice = [1,0])

neigh = KNeighborsClassifier(metric = 'precomputed', n_neighbors = 1, leaf_size = 70)
neigh.fit(X_train_new, y_train.ravel())
y_pred = neigh.predict(X_test_new)
print('Model: {}, \n Accuracy: {},\n RMSE: {}'.format(neigh,
                                                      accuracy_score(y_test, y_pred),
                                                      sqrt(mean_squared_error(y_test, y_pred))))

START fast...
You choose Hamming 1, Euclidean 2 and Hamming on pubchem2d 3...
Start Hamming su categorical...
Start Euclidean...
Start Hamming su Pubchem2d...
...FINISH
Model: KNeighborsClassifier(algorithm='auto', leaf_size=70, metric='precomputed',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform'), 
 Accuracy: 0.9008704690139016,
 RMSE: 0.31484842541467234
